# Examine spread for currency pairs

* NYC <==> Chicago trip, Google ping 1 way is 22.7ms, so a round trip takes about 50ms. A trading window of 20-50 ms is perhaps actionable, more than 50ms is definitely actionable.

* NYC <==> San Francisco, Google ping 1 way is 69.7ms, round trip is abour 140ms. A trading window of 70-140 ms is perhaps actionable, more than 150ms is definitely actionable.

* NYC <==> London is only a tiny bit slower than London.

* NYC <==> Hong Kong, Google ping is 247ms, a round trip is then about 500ms. Singapore is ~260ms one way.

See Ping time from NYC here
https://wondernetwork.com/pings/New%20York

In [ ]:
import sys
sys.path.append('..')

import time
import os
import re
from collections import defaultdict
import pprint as pp
import copy

from tqdm import tqdm
RE_FNAME = re.compile(r'LP\-(?P<idx>[0-9])\-STRM\-[0-9]\-(?P<pair>[A-Z]+).*\.csv')

from utils import GCStorage
from constants import CREDENTIAL_PATH, ALL_DAYS

import pandas as pd

import shutil
import csv
import numpy as np
from subprocess import call
import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import datetime
from dateutil.parser import parse
from pathlib import Path

from utils import GCStorage
from constants import *
from multiprocessing import Pool, Manager, Value


In [ ]:
pair = 'NZDUSD'

## Histogram for Trading Window. Statistics for Diff (in pip)

In [ ]:
def process_inversion_for_day(inversions):
    
    trading_windows = []
    diffs = []
    for inv in inversions:

        #print('Diff', inv["price diff"])
        #print(f'Starting spread: {round(inv["last valid bid"], 8)} - {round(inv["last valid ask"], 8)}')
        #print(f'Bid time: {inv["bid time"]}')
        #print(f'Ask time: {inv["ask time"]}')


        #print(start_time)

        #print('Last => Exit bid time', inv["last valid bid time"], inv["exit bid time"])
        #print('Last => Exit ask time', inv["last valid ask time"], inv["exit ask time"])

        start_time = max(inv["bid time"], inv["ask time"])
        last_valid = min(inv["last valid bid time"], inv["last valid ask time"])
        trading_window = int(round((last_valid - start_time).total_seconds() * 1000))

        if trading_window <= 0:
            continue
            
        #print(last_valid)

        #print('Trading Window (Miliseconds)', trading_window, f'{start_time} - {last_valid}')


        exit_time = min(inv["exit bid time"], inv["exit ask time"])
        #print(exit_time)


        #print('----')
        #print(f'Dif: {inv["price diff"]} pip\t{t_diff} milisec')
        #print(f'Bid: {inv["bid"]}\t{inv["bid time"]}')
        #print(f'Ask: {inv["ask"]}\t{inv["ask time"]}')
        #print('--------')
        
        trading_windows.append(trading_window)
        diffs.append(inv["price diff"])
    
    return trading_windows, diffs

In [ ]:
all_diffs = []
all_trading_windows = []
for day in ALL_DAYS:
    fname = f'inversions/{pair}/{pair}_{day}.pickle'
    if os.path.exists(fname):
        print(f'Loaded {day}')
        inversion = pickle.load(open(fname, 'rb'))
        print(len(inversion))

        trading_windows, diffs = process_inversion_for_day(inversion)
        all_trading_windows.extend(trading_windows)
        all_diffs.extend(diffs)

In [ ]:
uniques, counts = np.unique(all_diffs, return_counts=True)

for v, c in zip(uniques, counts):
    print(f'Diff(pip): {v}\tCount: {c}')
    

plt.figure(figsize=(12,5), dpi=100) # 640 x 450

axes = plt.subplot(111)

axes.hist(all_diffs, bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

axes.set_title(f'Histogram for Trading Diff ({pair})')
axes.set_xlabel('Difference (pip)')
axes.set_yscale('log', nonposy='clip')
# axes.xaxis.set_minor_formatter(FormatStrFormatter("%d"))
# plt.setp(axes.get_xticklabels(minor=True), visible=True)

plt.xlim(0, 1)
plt.grid()

In [ ]:
plt.figure(figsize=(12,5), dpi=100) # 640 x 450

axes = plt.subplot(111)

axes.set_xscale('log')
axes.set_yscale('log', nonposy='clip')
axes.hist(all_trading_windows, bins=[0, 1, 20, 50, 100, 250, 500, 1000])

axes.set_title(f'Histogram for Trading Windows ({pair})')
axes.set_xlabel('Trading Window in Miliseconds')
axes.set_xticks([0, 1, 20, 50, 100, 250, 500, 1000])
# axes.xaxis.set_minor_formatter(FormatStrFormatter("%d"))
# plt.setp(axes.get_xticklabels(minor=True), visible=True)

plt.xlim(0, 1500)
plt.grid()

## Bar Chart for "Tradable" Instances (NYC-Chicago) for Time of Day

In [ ]:
min_slot = 1.25 * 50

In [ ]:
def process_inversion_for_day(inversions):
    
    trading_windows = np.zeros(24)  # I guess there are 24 hours a day?
    diffs = []
    for inv in inversions:

        #print('Diff', inv["price diff"])
        #print(f'Starting spread: {round(inv["last valid bid"], 8)} - {round(inv["last valid ask"], 8)}')
        #print(f'Bid time: {inv["bid time"]}')
        #print(f'Ask time: {inv["ask time"]}')

        #print(start_time)

        #print('Last => Exit bid time', inv["last valid bid time"], inv["exit bid time"])
        #print('Last => Exit ask time', inv["last valid ask time"], inv["exit ask time"])

        start_time = max(inv["bid time"], inv["ask time"])
        last_valid = min(inv["last valid bid time"], inv["last valid ask time"])
        trading_window = int(round((last_valid - start_time).total_seconds() * 1000))

        if trading_window < min_slot:
            continue
            
        #print(last_valid)

        #print('Trading Window (Miliseconds)', trading_window, f'{start_time} - {last_valid}')


        exit_time = min(inv["exit bid time"], inv["exit ask time"])
        #print(exit_time)


        #print('----')
        #print(f'Dif: {inv["price diff"]} pip\t{t_diff} milisec')
        #print(f'Bid: {inv["bid"]}\t{inv["bid time"]}')
        #print(f'Ask: {inv["ask"]}\t{inv["ask time"]}')
        #print('--------')
        
        # trading_windows.append(trading_window)
        trading_windows[inv["bid time"].hour] += 1
        diffs.append(inv["price diff"])
    
    return trading_windows, diffs

In [ ]:
all_hours = np.zeros(24)
all_diffs = []
for day in ALL_DAYS:
    fname = f'inversions/{pair}/{pair}_{day}.pickle'
    if os.path.exists(fname):
        print(f'Loaded {day}')
        inversion = pickle.load(open(fname, 'rb'))
        print(len(inversion))

        hours, diffs = process_inversion_for_day(inversion)
        all_hours += hours
        all_diffs.extend(diffs)

In [ ]:
uniques, counts = np.unique(all_diffs, return_counts=True)

profit = 0
for v, c in zip(uniques, counts):
    print(f'Diff(pip): {v}\tCount: {c}')
    profit += 1000 * v
print(f'Potential profit: {profit}')
    

plt.figure(figsize=(12,5), dpi=100) # 640 x 450

axes = plt.subplot(111)

axes.hist(all_diffs, bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

axes.set_title(f'Histogram for Tradable Diff ({pair})(window > {min_slot}ms)')
axes.set_xlabel('Difference (pip)')
axes.set_yscale('log', nonposy='clip')
# axes.xaxis.set_minor_formatter(FormatStrFormatter("%d"))
# plt.setp(axes.get_xticklabels(minor=True), visible=True)

plt.xlim(0, 1)
plt.grid()

In [ ]:
plt.figure(figsize=(12,5), dpi=100) # 640 x 450

axes = plt.subplot(111)

#axes.set_xscale('log')
#axes.set_yscale('log', nonposy='clip')
axes.bar(np.array(range(0, 24)), all_hours, align='edge')

axes.set_title(f'Bar Chart for Trading Windows ({pair})(window > {min_slot}ms)')
axes.set_xlabel('Time of Day')
axes.set_ylabel('Number of Tradable Opportunities')
axes.set_xticks([0, 4, 8, 12, 16, 20, 24])

# plt.xlim(0, 1500)
plt.grid()